# Complete mCODE Translation Workflow with Concurrency

This notebook demonstrates the complete mCODE translation pipeline using individual component scripts with **concurrent processing**.

**Prerequisites:**
- Python environment with all dependencies installed
- LLM API keys configured (OpenAI, etc.)
- CORE Memory API key is configured (optional, for storage)
- All dependencies are installed

**What this notebook does:**
1. **Concurrent Fetching**: Fetches clinical trial data using multiple workers
2. **Data Download**: Downloads synthetic patient data archives
3. **Patient Fetching**: Extracts synthetic patients from downloaded archives
4. **Concurrent Optimization**: Tests AI model combinations in parallel
5. **Concurrent Processing**: Converts data to mCODE format with multiple workers
6. **Concurrent Summarization**: Generates summaries and stores in CORE Memory

**Concurrency Features:**
- **Fetcher Pool**: 8 workers for concurrent API calls
- **Processor Pool**: 12 workers for parallel data processing
- **Optimizer Pool**: 15 workers for parallel model testing
- **Task Queues**: Priority-based task execution with progress tracking

**Command Chaining Examples:**

This notebook demonstrates different approaches to chaining commands:

- **File-based**: Save intermediate results to files (current approach)
- **Pipe-based**: Chain commands using `|` for streaming data
- **Combined**: Mix files and pipes for optimal performance

**Example Command Chains:**
```bash
# Complete pipeline: fetcher → processor → summarizer
python -m src.cli.patients_fetcher --archive breast_cancer_10_years | \
    python -m src.cli.patients_processor | \
    python -m src.cli.patients_summarizer --ingest

# File-based approach with intermediate files
python -m src.cli.patients_fetcher --archive breast_cancer_10_years --out raw_patients.ndjson
python -m src.cli.patients_processor --in raw_patients.ndjson --out mcode_patients.ndjson
python -m src.cli.patients_summarizer --in mcode_patients.ndjson --ingest

# Parallel processing with background jobs
python -m src.cli.patients_fetcher --archive breast_cancer_10_years | \
    python -m src.cli.patients_processor | \
    python -m src.cli.patients_summarizer --ingest &

python -m src.cli.trials_fetcher --condition "breast cancer" --limit 5 | \
    python -m src.cli.trials_processor | \
    python -m src.cli.trials_summarizer --ingest &
wait
```

## Step 1: Environment Setup

Configure API keys and verify the environment is ready.

In [ ]:
# Configure environment and API keys
import os
import sys
from pathlib import Path

# Change to project root directory
current_dir = Path.cwd()
if current_dir.name == 'examples':
    project_root = current_dir.parent
    os.chdir(project_root)
    print(f"📁 Changed working directory to: {project_root}")
else:
    project_root = current_dir

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("✅ Added project root to Python path")

# Configure API keys (replace with your actual keys)
os.environ['CLINICAL_TRIALS_API_KEY'] = 'your_clinical_trials_api_key_here'
os.environ['COREAI_API_KEY'] = 'your_core_memory_api_key_here'

print("✅ API keys configured")
print("✅ Environment ready")
print(f"📍 Current working directory: {Path.cwd()}")

## Step 2: Fetch Clinical Trial Data

Download clinical trial data from ClinicalTrials.gov for breast cancer studies.

**What are clinical trials?** Research studies that test new treatments on human volunteers.

In [ ]:
# Fetch 5 breast cancer clinical trials from ClinicalTrials.gov
# This provides real-world treatment data for mCODE conversion
# Using 8 concurrent workers for faster fetching
!python -m src.cli.trials_fetcher --condition "breast cancer" --limit 5 --out raw_trials.ndjson --workers 8 --verbose

## Step 3: Download Data Archives

Download the synthetic patient data archives needed for the workflow.

**Why download archives?** The patient fetcher requires local archives containing synthetic FHIR data.

In [ ]:
# Download all available synthetic patient data archives
# This provides comprehensive test data for the workflow
!python -m scripts.download_data --all

## Step 4: Fetch Synthetic Patient Data

Download synthetic patient records that mimic real breast cancer patients.

**Why synthetic patients?** They protect patient privacy while providing realistic data for testing.

In [ ]:
# Fetch 5 synthetic breast cancer patients from the 10-year archive
# These are artificially generated but realistic patient records
!python -m src.cli.patients_fetcher --archive breast_cancer_10_years --limit 5 --out raw_patients.ndjson --verbose

## Step 5: Optimize AI Model Parameters

Find the best AI model and prompt combination for processing breast cancer data.

### What is AI Model Optimization?

**AI model optimization** is the process of finding the best combination of AI model and prompt template for a specific task. Just like different cars perform better on different terrains, different AI models excel at different types of medical data processing.

### Why is Optimization Needed?

1. **Model Performance Varies**: Some models are better at understanding clinical trial protocols, while others excel at patient data analysis
2. **Cost vs. Quality Trade-offs**: More expensive models (like GPT-4) may not always provide better results than cost-effective alternatives
3. **Prompt Sensitivity**: The same model can produce very different results with different prompt instructions
4. **Data Type Specificity**: Models optimized for code may not work well with medical narratives, and vice versa

### How Optimization Works

The optimizer uses **cross-validation** to test multiple model+prompt combinations:

1. **Splits your data** into training and validation sets
2. **Tests each combination** on the validation data
3. **Scores performance** using mCODE compliance metrics
4. **Ranks combinations** by average cross-validation score
5. **Saves the best configuration** for production use

### What Gets Optimized?

- **Model Selection**: GPT-4, GPT-4o, DeepSeek-Coder, etc.
- **Prompt Templates**: Different instruction styles for mCODE conversion
- **Performance Metrics**: mCODE compliance, mapping accuracy, processing speed

### Benefits of Optimization

- **20-50% better accuracy** for your specific data type
- **Cost savings** by using the most efficient model for the job
- **Consistent results** across different data sources
- **Future-proofing** as new models become available

**Note:** The optimizer only works with existing files, never fetches new data via APIs.

In [ ]:
# Test multiple model+prompt combinations using the fetched trial data
# This finds the best configuration for processing breast cancer trials
# The optimizer only works with files, never uses APIs directly
# Using 15 concurrent workers for parallel optimization
!python -m src.cli.trials_optimizer --trials-file raw_trials.ndjson --cv-folds 3 --prompts direct_mcode_evidence_based_concise,direct_mcode_minimal --models gpt-4,gpt-4o,deepseek-coder,deepseek-chat --max-combinations 8 --save-config optimal_config.json --workers 15 --verbose

### 4a: Load Optimal Configuration

Read the optimization results and store the best model/prompt combination for later use.

In [ ]:
# Load the optimal AI configuration found by the optimizer
import json
from pathlib import Path

config_file = Path('optimal_config.json')
if config_file.exists():
    with open(config_file, 'r') as f:
        optimal_config = json.load(f)
    
    # Store the best settings for use in later steps
    BEST_MODEL = optimal_config['optimal_settings']['model']
    BEST_PROMPT = optimal_config['optimal_settings']['prompt']
    
    print(f"🎯 Optimal configuration:")
    print(f"   Model: {BEST_MODEL}")
    print(f"   Prompt: {BEST_PROMPT}")
    print(f"   CV score: {optimal_config['optimal_settings']['cv_score']:.3f}")
else:
    # Fallback to defaults if optimization failed
    BEST_MODEL = 'deepseek-coder'
    BEST_PROMPT = 'direct_mcode_evidence_based_concise'
    print("⚠️  Using default configuration (optimization may have failed)")
    print(f"   Model: {BEST_MODEL}")
    print(f"   Prompt: {BEST_PROMPT}")

## Step 6: Process Trials to mCODE Format

Convert raw clinical trial data to structured mCODE format using the optimized AI model.

**What is mCODE?** Minimal Common Oncology Data Elements - standardized format for cancer data.

In [ ]:
# Convert raw trial data to mCODE format using the best AI model
# This structures the trial information according to mCODE standards
# Using 12 concurrent workers for faster processing
!python -m src.cli.trials_processor raw_trials.ndjson --out mcode_trials.ndjson --model {BEST_MODEL} --prompt {BEST_PROMPT} --workers 12 --verbose

## Step 7: Process Patients to mCODE Format

Convert raw patient data to structured mCODE format.

**Patient processing** focuses on individual medical histories, diagnoses, and treatments.

In [ ]:
# Convert raw patient data to mCODE format
# This structures patient medical histories according to mCODE standards
# Using 12 concurrent workers for faster processing
!python -m src.cli.patients_processor --in raw_patients.ndjson --out mcode_patients.ndjson --workers 12 --verbose

## Step 8: Generate Summaries and Store in CORE Memory

Create human-readable summaries of all processed data and store permanently.

**CORE Memory** provides persistent storage and search capabilities for all processed data.

### 7a: Generate Trial Summaries

Create readable summaries of the clinical trials and store them in CORE Memory.

In [ ]:
# Generate human-readable summaries of the mCODE trial data
# Store summaries in CORE Memory for permanent access and search
!python -m src.cli.trials_summarizer --in mcode_trials.ndjson --model {BEST_MODEL} --ingest --verbose

### 7b: Generate Patient Summaries

Create readable summaries of the patient data and store them in CORE Memory.

In [ ]:
# Generate human-readable summaries of the mCODE patient data
# Store summaries in CORE Memory for permanent access and search
!python -m src.cli.patients_summarizer --in mcode_patients.ndjson --ingest --verbose

## Step 9: Verify Results

Check that all files were created successfully and show a summary of the work completed.

In [ ]:
# Check which files were created and their sizes
import os
from pathlib import Path

files_to_check = [
    'optimal_config.json',
    'raw_trials.ndjson',
    'raw_patients.ndjson',
    'mcode_trials.ndjson',
    'mcode_patients.ndjson'
]

print("📁 Generated files:")
for filename in files_to_check:
    file_path = Path(filename)
    if file_path.exists():
        size = file_path.stat().st_size
        print(f"   ✅ {filename} ({size} bytes)")
    else:
        print(f"   ❌ {filename} (not found)")

print("\n🎉 mCODE translation workflow completed!")
print("All data has been processed and stored in CORE Memory.")

## Summary

This notebook successfully completed the complete mCODE translation workflow with **concurrent processing**:

✅ **Fetched** 5 clinical trials for breast cancer
✅ **Downloaded** synthetic patient data archives
✅ **Extracted** 5 synthetic patients from archives
✅ **Optimized** AI model parameters using existing files (no API calls)
✅ **Processed** all data into structured mCODE format
✅ **Generated** human-readable summaries
✅ **Stored** everything permanently in CORE Memory

### Generated Files:
- `raw_trials.ndjson` - Original clinical trial data
- `raw_patients.ndjson` - Original synthetic patient data
- `mcode_trials.ndjson` - Structured trial mCODE data
- `mcode_patients.ndjson` - Structured patient mCODE data
- `optimal_config.json` - Best AI model configuration

### Key Features:
- **Concurrent fetching**: 8 workers for API calls
- **Concurrent optimization**: 15 workers for model testing
- **Concurrent processing**: 12 workers for data conversion
- **File-only optimization**: Optimizer never uses APIs, only cross-validates existing files
- **Component-based**: Uses individual scripts for each step
- **Educational**: Each command is explained with context

### Next Steps:
- Explore the generated mCODE data
- Search CORE Memory for specific information
- Run the workflow with different conditions or parameters
- Integrate the processed data into other applications

## Alternative: Command Chaining Approaches

### Pipe-Based Workflow (Memory Efficient)

For memory-constrained environments or streaming processing:

```bash
# Stream trials directly to processing
python -m src.cli.trials_fetcher --condition "breast cancer" --limit 5 --workers 8 \
    | python -m src.cli.trials_processor --out mcode_trials.ndjson --workers 12

# Stream patients directly to processing
python -m src.cli.patients_fetcher --archive breast_cancer_10_years \
    | python -m src.cli.patients_processor --out mcode_patients.ndjson --workers 12

# Continue with optimization and summarization...
```

### Parallel Processing with Background Jobs

For maximum concurrency with multiple data streams:

```bash
# Start background processing jobs
python -m src.cli.trials_fetcher --condition "breast cancer" --limit 5 --out raw_trials.ndjson --workers 8 &
python -m src.cli.patients_fetcher --archive breast_cancer_10_years --out raw_patients.ndjson &

# Wait for data fetching to complete
wait

# Process both streams in parallel
python -m src.cli.trials_processor raw_trials.ndjson --out mcode_trials.ndjson --workers 12 &
python -m src.cli.patients_processor raw_patients.ndjson --out mcode_patients.ndjson --workers 12 &

# Wait for processing to complete
wait

# Continue with optimization and summarization...
```

### Hybrid Approach (Recommended)

Combine files and pipes for optimal performance:

```bash
# Use files for complex operations, pipes for simple transformations
python -m src.cli.trials_fetcher --condition "breast cancer" --limit 5 --out raw_trials.ndjson --workers 8
python -m src.cli.patients_fetcher --archive breast_cancer_10_years | \
    python -m src.cli.patients_processor --out mcode_patients.ndjson --workers 12

# Optimization works best with files
python -m src.cli.trials_optimizer --trials-file raw_trials.ndjson --cv-folds 3 \
    --prompts direct_mcode_evidence_based_concise,direct_mcode_minimal \
    --models deepseek-coder,gpt-4o --save-config optimal_config.json --workers 15

# Extract optimal settings and continue
# ... (rest of workflow)
```